<a href="https://colab.research.google.com/github/prp20/tensorflow_learning/blob/main/NLP_ZTM_Course.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#check for nvidia smi
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-bd50b402-86e9-5515-863d-46ff7a13ccc0)


In [2]:
# GEt helper functions
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2022-10-22 14:08:52--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-10-22 14:08:52 (109 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
# Get dataset from Kaggle
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

#Unzip data
unzip_data('nlp_getting_started.zip')

--2022-10-22 14:08:55--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.195.128, 173.194.202.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.009s  

2022-10-22 14:08:55 (63.7 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [4]:
# Import dataset using pandas

import pandas as pd
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [5]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
#shuffle data
train_df_shuffled = train_data.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
# Create test train split
from sklearn.model_selection import train_test_split

train_sent, val_sent, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(), train_df_shuffled['target'].to_numpy(), test_size = 0.1, random_state = 42)

In [8]:
len(train_sent), len(train_labels), len(val_sent), len(val_labels)

(6851, 6851, 762, 762)

In [9]:
train_sent[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

In [10]:
 import tensorflow as tf
 from tensorflow.keras.layers import TextVectorization

In [11]:
max_vocab_length = 10000
max_length = 15
text_vectorizer = TextVectorization(max_tokens=max_vocab_length, 
                                    output_mode="int", 
                                    output_sequence_length=max_length)

In [12]:
text_vectorizer.adapt(train_sent)

In [13]:
import random
random_sentence = random.choice(train_sent)
print(f"Random Sentence: {random_sentence}\n")
print(f"Vectorized version of random sentence: {text_vectorizer(random_sentence)}")

Random Sentence: Is Stuart Broad the Prime Minister yet. Best thing in Sport I have seen for years that. The Aussies look devastated. Bless ??????

Vectorized version of random sentence: [   9 2275 2176    2 2017 2043  522  149  498    4 1759    8   24  834
   10]


In [14]:
words_in_vocab = text_vectorizer.get_vocabulary()

In [15]:
top_5_words = words_in_vocab[:5]
least_5_words = words_in_vocab[-5:]

In [16]:
top_5_words

['', '[UNK]', 'the', 'a', 'in']

In [17]:
least_5_words

['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']

In [18]:
# Creating an embedding layer
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length, output_dim=128, input_length=max_length)

#Various Models in this exp
  * Naive bayes based TF-IDF encoder
  * Feed Forward Neural Network
  * LSTM
  * GRU
  * Bi directional RNN
  * 1D CNN
  * Tensorflow pretrained feature extractor
  * Tensorflow pretrained feature extractor with 10% data



In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [20]:
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ('clf', MultinomialNB())
])

model_0.fit(train_sent, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [21]:
#evaluate the model
baseline_score = model_0.score(val_sent, val_labels)
baseline_score
pred_results = model_0.predict(val_sent)

In [22]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  model_accuracy = accuracy_score(y_true, y_pred) *100
  model_precision, model_recall, model_f1score, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"Accuracy": model_accuracy, "Precision": model_precision, "Recall value": model_recall, "F1-Score": model_f1score}

  return model_results

In [23]:
# get baseline results
baseline_results = calculate_results(val_labels, pred_results)

In [24]:
baseline_results

{'Accuracy': 79.26509186351706,
 'Precision': 0.8111390004213173,
 'Recall value': 0.7926509186351706,
 'F1-Score': 0.7862189758049549}

## Model 1: Basic Model

In [25]:
from helper_functions import create_tensorboard_callback
SAVE_DIR = "model_logs"

In [26]:
from IPython.utils import text
from tensorflow.keras import layers

input = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(input)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_1 = tf.keras.Model(input, outputs, name = "NLP_model_1")

In [27]:
model_1.summary()

Model: "NLP_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non

In [28]:
model_1.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

In [29]:
history_1 = model_1.fit(train_sent, train_labels, epochs=10, validation_data =(val_sent, val_labels))

Epoch 1/10
215/215 [==============================] - 4s 5ms/step - loss: 0.6114 - accuracy: 0.6897 - val_loss: 0.5364 - val_accuracy: 0.7546
Epoch 2/10
215/215 [==============================] - 1s 4ms/step - loss: 0.4419 - accuracy: 0.8173 - val_loss: 0.4734 - val_accuracy: 0.7835
Epoch 3/10
215/215 [==============================] - 1s 4ms/step - loss: 0.3461 - accuracy: 0.8603 - val_loss: 0.4573 - val_accuracy: 0.7966
Epoch 4/10
215/215 [==============================] - 1s 4ms/step - loss: 0.2835 - accuracy: 0.8915 - val_loss: 0.4655 - val_accuracy: 0.7913
Epoch 5/10
215/215 [==============================] - 1s 4ms/step - loss: 0.2379 - accuracy: 0.9113 - val_loss: 0.4817 - val_accuracy: 0.7769
Epoch 6/10
215/215 [==============================] - 1s 4ms/step - loss: 0.2013 - accuracy: 0.9276 - val_loss: 0.5011 - val_accuracy: 0.7743
Epoch 7/10
215/215 [==============================] - 1s 4ms/step - loss: 0.1721 - accuracy: 0.9422 - val_loss: 0.5307 - val_accuracy: 0.7848
Epoch 

In [30]:
model_1.evaluate(val_sent, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.6116 - accuracy: 0.7769


[0.6115506887435913, 0.7769029140472412]

In [31]:
pred_data = model_1.predict(val_sent)

24/24 [==============================] - 0s 2ms/step


In [32]:
pred_data = tf.squeeze(tf.round(pred_data))
pred_data

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [33]:
model_1_results = calculate_results(val_labels, pred_data)
model_1_results

{'Accuracy': 77.69028871391076,
 'Precision': 0.7781874595396938,
 'Recall value': 0.7769028871391076,
 'F1-Score': 0.7751243211017478}

In [34]:
baseline_results

{'Accuracy': 79.26509186351706,
 'Precision': 0.8111390004213173,
 'Recall value': 0.7926509186351706,
 'F1-Score': 0.7862189758049549}

## Model 2: LSTM

In [35]:
from tensorflow.keras import layers

input = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(input)
x = embedding(x)
x = layers.LSTM(units=64, return_sequences=True)(x)
x = layers.LSTM(64)(x)
x = layers.Dense(64, activation='relu')(x)
output = layers.Dense(1, activation='sigmoid')(x)

model_2 = tf.keras.Model(input, output, name="nlp_model_2_LSTM")

In [36]:
model_2.summary()

Model: "nlp_model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 15, 64)            49408     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                  

In [37]:
model_2.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics = ['accuracy'])
history_2 = model_2.fit(train_sent, train_labels, epochs=10, validation_data = (val_sent, val_labels))

Epoch 1/10
215/215 [==============================] - 13s 25ms/step - loss: 0.1428 - accuracy: 0.9553 - val_loss: 0.8157 - val_accuracy: 0.7782
Epoch 2/10
215/215 [==============================] - 4s 17ms/step - loss: 0.0886 - accuracy: 0.9661 - val_loss: 0.7467 - val_accuracy: 0.7717
Epoch 3/10
215/215 [==============================] - 2s 10ms/step - loss: 0.0811 - accuracy: 0.9685 - val_loss: 1.0694 - val_accuracy: 0.7703
Epoch 4/10
215/215 [==============================] - 2s 8ms/step - loss: 0.0688 - accuracy: 0.9730 - val_loss: 1.0930 - val_accuracy: 0.7690
Epoch 5/10
215/215 [==============================] - 2s 8ms/step - loss: 0.0605 - accuracy: 0.9753 - val_loss: 1.1451 - val_accuracy: 0.7717
Epoch 6/10
215/215 [==============================] - 2s 8ms/step - loss: 0.0519 - accuracy: 0.9768 - val_loss: 1.6443 - val_accuracy: 0.7690
Epoch 7/10
215/215 [==============================] - 2s 8ms/step - loss: 0.0509 - accuracy: 0.9768 - val_loss: 1.2514 - val_accuracy: 0.7677
Ep

In [38]:
model_2.evaluate(val_sent, val_labels)

24/24 [==============================] - 0s 4ms/step - loss: 1.3383 - accuracy: 0.7572


[1.3383471965789795, 0.7572178244590759]

In [39]:
pred_data_2 = tf.squeeze(tf.round(model_2.predict(val_sent)))
pred_data_2

24/24 [==============================] - 1s 4ms/step


<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0.,
       0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0.,
       0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0

In [40]:
model_2_results = calculate_results(val_labels, pred_data_2)
model_2_results

{'Accuracy': 75.7217847769029,
 'Precision': 0.7573294948869645,
 'Recall value': 0.7572178477690289,
 'F1-Score': 0.7558283776496815}

## Model 3: GRU

In [41]:
from tensorflow.keras import layers

input = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(input)
x = embedding(x)
x = layers.GRU(64, return_sequences=True)(x)
x = layers.LSTM(64, return_sequences=True)(x)
x = layers.GRU(64)(x)
x = layers.Dense(64, activation='relu')(x)
output = layers.Dense(1, activation='sigmoid')(x)

model_3 = tf.keras.Model(input, output, name="nlp_model_3_GRU")

In [42]:
model_3.summary()

Model: "nlp_model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 15, 64)            37248     
                                                                 
 lstm_2 (LSTM)               (None, 15, 64)            33024     
                                                                 
 gru_1 (GRU)                 (None, 64)                24960     
                                                   

In [43]:
model_3.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics = ['accuracy'])
history_3 = model_3.fit(train_sent, train_labels, epochs=10, validation_data=(val_sent, val_labels))

Epoch 1/10
215/215 [==============================] - 7s 13ms/step - loss: 0.1195 - accuracy: 0.9661 - val_loss: 0.9597 - val_accuracy: 0.7559
Epoch 2/10
215/215 [==============================] - 2s 9ms/step - loss: 0.0536 - accuracy: 0.9771 - val_loss: 1.3341 - val_accuracy: 0.7585
Epoch 3/10
215/215 [==============================] - 2s 9ms/step - loss: 0.0456 - accuracy: 0.9781 - val_loss: 1.5088 - val_accuracy: 0.7677
Epoch 4/10
215/215 [==============================] - 2s 9ms/step - loss: 0.0426 - accuracy: 0.9799 - val_loss: 1.5821 - val_accuracy: 0.7651
Epoch 5/10
215/215 [==============================] - 2s 9ms/step - loss: 0.0448 - accuracy: 0.9796 - val_loss: 1.7309 - val_accuracy: 0.7651
Epoch 6/10
215/215 [==============================] - 2s 9ms/step - loss: 0.0437 - accuracy: 0.9799 - val_loss: 1.4607 - val_accuracy: 0.7690
Epoch 7/10
215/215 [==============================] - 2s 9ms/step - loss: 0.0419 - accuracy: 0.9791 - val_loss: 1.4668 - val_accuracy: 0.7612
Epoch

In [44]:
model_3.evaluate(val_sent, val_labels)

24/24 [==============================] - 0s 5ms/step - loss: 1.9926 - accuracy: 0.7612


[1.9926307201385498, 0.76115483045578]

In [45]:
pred_data_3 = tf.squeeze(tf.round(model_3.predict(val_sent)))
model_3_results = calculate_results(val_labels, pred_data_3)

24/24 [==============================] - 1s 4ms/step


In [46]:
model_3_results

{'Accuracy': 76.11548556430446,
 'Precision': 0.7622953452640989,
 'Recall value': 0.7611548556430446,
 'F1-Score': 0.7591215732608759}

In [47]:
baseline_results

{'Accuracy': 79.26509186351706,
 'Precision': 0.8111390004213173,
 'Recall value': 0.7926509186351706,
 'F1-Score': 0.7862189758049549}

In [48]:
from tensorflow.keras import layers

input = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(input)
x = embedding(x)
x = layers.GRU(64, return_sequences=True)(x)
x = layers.LSTM(64, return_sequences=True)(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(64, activation='relu')(x)
output = layers.Dense(1, activation='sigmoid')(x)

model_3_1 = tf.keras.Model(input, output, name="nlp_model_3_GRU_1")
model_3_1.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics = ['accuracy'])
history_3_1 = model_3_1.fit(train_sent, train_labels, epochs=10, validation_data=(val_sent, val_labels))

Epoch 1/10
215/215 [==============================] - 5s 12ms/step - loss: 0.1394 - accuracy: 0.9555 - val_loss: 0.9209 - val_accuracy: 0.7625
Epoch 2/10
215/215 [==============================] - 2s 8ms/step - loss: 0.0512 - accuracy: 0.9778 - val_loss: 1.1564 - val_accuracy: 0.7677
Epoch 3/10
215/215 [==============================] - 2s 8ms/step - loss: 0.0413 - accuracy: 0.9809 - val_loss: 1.4909 - val_accuracy: 0.7690
Epoch 4/10
215/215 [==============================] - 2s 8ms/step - loss: 0.0378 - accuracy: 0.9829 - val_loss: 1.4850 - val_accuracy: 0.7625
Epoch 5/10
215/215 [==============================] - 2s 8ms/step - loss: 0.0426 - accuracy: 0.9788 - val_loss: 1.7011 - val_accuracy: 0.7572
Epoch 6/10
215/215 [==============================] - 2s 8ms/step - loss: 0.0408 - accuracy: 0.9796 - val_loss: 1.6554 - val_accuracy: 0.7651
Epoch 7/10
215/215 [==============================] - 2s 8ms/step - loss: 0.0365 - accuracy: 0.9807 - val_loss: 1.9913 - val_accuracy: 0.7651
Epoch

In [49]:
model_3_1.evaluate(val_sent, val_labels)

24/24 [==============================] - 0s 5ms/step - loss: 1.7769 - accuracy: 0.7493


[1.7768847942352295, 0.7493438124656677]

In [50]:
pred_data_3_1 = tf.squeeze(tf.round(model_3_1.predict(val_sent)))
model_3_1_results = calculate_results(val_labels, pred_data_3_1)
model_3_1_results

24/24 [==============================] - 1s 3ms/step


{'Accuracy': 74.93438320209974,
 'Precision': 0.7494029493702923,
 'Recall value': 0.7493438320209974,
 'F1-Score': 0.7493720973122503}

## Model 4: Bi-Directional LSTM

In [51]:
from tensorflow.keras import layers

input = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(input)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.GRU(64, return_sequences=True))(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(64, activation='relu')(x)
output = layers.Dense(1, activation='sigmoid')(x)

model_4 = tf.keras.Model(input, output, name="nlp_model_4_BI_LSTM")
model_4.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics = ['accuracy'])
history_4 = model_4.fit(train_sent, train_labels, epochs=10, validation_data=(val_sent, val_labels))

Epoch 1/10
215/215 [==============================] - 9s 18ms/step - loss: 0.0794 - accuracy: 0.9753 - val_loss: 1.3739 - val_accuracy: 0.7415
Epoch 2/10
215/215 [==============================] - 3s 12ms/step - loss: 0.0395 - accuracy: 0.9797 - val_loss: 1.5318 - val_accuracy: 0.7546
Epoch 3/10
215/215 [==============================] - 2s 12ms/step - loss: 0.0327 - accuracy: 0.9826 - val_loss: 1.8359 - val_accuracy: 0.7572
Epoch 4/10
215/215 [==============================] - 3s 12ms/step - loss: 0.0383 - accuracy: 0.9810 - val_loss: 1.4504 - val_accuracy: 0.7572
Epoch 5/10
215/215 [==============================] - 3s 14ms/step - loss: 0.0323 - accuracy: 0.9837 - val_loss: 1.9261 - val_accuracy: 0.7493
Epoch 6/10
215/215 [==============================] - 3s 13ms/step - loss: 0.0302 - accuracy: 0.9835 - val_loss: 2.1293 - val_accuracy: 0.7625
Epoch 7/10
215/215 [==============================] - 3s 12ms/step - loss: 0.0343 - accuracy: 0.9838 - val_loss: 1.9120 - val_accuracy: 0.7572

In [52]:
model_4.evaluate(val_sent, val_labels)

24/24 [==============================] - 0s 6ms/step - loss: 2.0683 - accuracy: 0.7559


[2.0682501792907715, 0.7559055089950562]

In [53]:
pred_data_4 = tf.squeeze(tf.round(model_4.predict(val_sent)))
model_4_results = calculate_results(val_labels, pred_data_4)
model_4_results

24/24 [==============================] - 2s 5ms/step


{'Accuracy': 75.59055118110236,
 'Precision': 0.7556483058194802,
 'Recall value': 0.7559055118110236,
 'F1-Score': 0.7548850741589771}

## Model 5: CNN in NLP

In [54]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(64, 5, 1, activation='relu',padding="same")(x)
x = layers.GlobalMaxPool1D()(x)
x = layers.Dense(64, activation='relu')(x)
output = layers.Dense(1, activation="sigmoid")(x)

model_5 = tf.keras.Model(inputs, output, name="NLP_model_5_CNN")

In [55]:
model_5.summary()

Model: "NLP_model_5_CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d (Conv1D)             (None, 15, 64)            41024     
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense_9 (Dense)             (None, 64)            

In [56]:
model_5.compile(loss="binary_crossentropy", optimizer = tf.keras.optimizers.Adam(), metrics = ['accuracy'])
history_5 = model_5.fit(train_sent, train_labels, epochs=10, validation_data = (val_sent, val_labels))

Epoch 1/10
215/215 [==============================] - 6s 6ms/step - loss: 0.0968 - accuracy: 0.9721 - val_loss: 0.9162 - val_accuracy: 0.7638
Epoch 2/10
215/215 [==============================] - 1s 5ms/step - loss: 0.0481 - accuracy: 0.9794 - val_loss: 1.1618 - val_accuracy: 0.7625
Epoch 3/10
215/215 [==============================] - 1s 5ms/step - loss: 0.0377 - accuracy: 0.9812 - val_loss: 1.4708 - val_accuracy: 0.7598
Epoch 4/10
215/215 [==============================] - 1s 5ms/step - loss: 0.0322 - accuracy: 0.9832 - val_loss: 1.5618 - val_accuracy: 0.7598
Epoch 5/10
215/215 [==============================] - 1s 5ms/step - loss: 0.0304 - accuracy: 0.9839 - val_loss: 1.8027 - val_accuracy: 0.7520
Epoch 6/10
215/215 [==============================] - 1s 5ms/step - loss: 0.0293 - accuracy: 0.9831 - val_loss: 1.8148 - val_accuracy: 0.7546
Epoch 7/10
215/215 [==============================] - 1s 5ms/step - loss: 0.0288 - accuracy: 0.9845 - val_loss: 2.0396 - val_accuracy: 0.7559
Epoch 

In [57]:
model_5.evaluate(val_sent, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 2.2988 - accuracy: 0.7585


[2.2987725734710693, 0.7585301995277405]

In [58]:
pred_data_5 = tf.squeeze(tf.round(model_5.predict(val_sent)))
model_5_results = calculate_results(val_labels, pred_data_5)

24/24 [==============================] - 0s 2ms/step


In [59]:
model_5_results

{'Accuracy': 75.8530183727034,
 'Precision': 0.7588734100545125,
 'Recall value': 0.7585301837270341,
 'F1-Score': 0.7569752644555794}

## Model 6: NLP using USE pre-trained model

In [60]:
from tensorflow.keras import layers
import tensorflow_hub as hub

sentence_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4", input_shape=[], dtype="string", trainable=False)



In [63]:
model_6 = tf.keras.models.Sequential([
    sentence_layer,
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation="sigmoid")
], name="nlp_model_6_use")

model_6.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
model_6.fit(train_sent, train_labels, epochs=10, validation_data=(val_sent, val_labels))

Epoch 1/10
215/215 [==============================] - 7s 14ms/step - loss: 0.5043 - accuracy: 0.7872 - val_loss: 0.4556 - val_accuracy: 0.8018
Epoch 2/10
215/215 [==============================] - 3s 12ms/step - loss: 0.4155 - accuracy: 0.8148 - val_loss: 0.4378 - val_accuracy: 0.8110
Epoch 3/10
215/215 [==============================] - 3s 12ms/step - loss: 0.4010 - accuracy: 0.8227 - val_loss: 0.4356 - val_accuracy: 0.8136
Epoch 4/10
215/215 [==============================] - 3s 12ms/step - loss: 0.3934 - accuracy: 0.8262 - val_loss: 0.4309 - val_accuracy: 0.8136
Epoch 5/10
215/215 [==============================] - 2s 12ms/step - loss: 0.3865 - accuracy: 0.8318 - val_loss: 0.4269 - val_accuracy: 0.8189
Epoch 6/10
215/215 [==============================] - 3s 12ms/step - loss: 0.3785 - accuracy: 0.8348 - val_loss: 0.4251 - val_accuracy: 0.8176
Epoch 7/10
215/215 [==============================] - 2s 11ms/step - loss: 0.3726 - accuracy: 0.8355 - val_loss: 0.4251 - val_accuracy: 0.8176

In [64]:
pred_data_6 = tf.squeeze(tf.round(model_6.predict(val_sent)))
model_6_results = calculate_results(val_labels, pred_data_6)

24/24 [==============================] - 1s 9ms/step


In [65]:
model_6_results

{'Accuracy': 82.1522309711286,
 'Precision': 0.8244694164885018,
 'Recall value': 0.821522309711286,
 'F1-Score': 0.8199037266090726}

In [66]:
baseline_results

{'Accuracy': 79.26509186351706,
 'Precision': 0.8111390004213173,
 'Recall value': 0.7926509186351706,
 'F1-Score': 0.7862189758049549}

In [67]:
model_6_1 = tf.keras.models.Sequential([
    sentence_layer,
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="sigmoid")
], name="nlp_model_6_use_1")

model_6_1.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
model_6_1.fit(train_sent, train_labels, epochs=10, validation_data=(val_sent, val_labels))

Epoch 1/10
215/215 [==============================] - 4s 13ms/step - loss: 0.5032 - accuracy: 0.7897 - val_loss: 0.4480 - val_accuracy: 0.7992
Epoch 2/10
215/215 [==============================] - 4s 17ms/step - loss: 0.4137 - accuracy: 0.8161 - val_loss: 0.4397 - val_accuracy: 0.8123
Epoch 3/10
215/215 [==============================] - 3s 15ms/step - loss: 0.4013 - accuracy: 0.8231 - val_loss: 0.4328 - val_accuracy: 0.8097
Epoch 4/10
215/215 [==============================] - 2s 12ms/step - loss: 0.3925 - accuracy: 0.8283 - val_loss: 0.4319 - val_accuracy: 0.8123
Epoch 5/10
215/215 [==============================] - 2s 12ms/step - loss: 0.3865 - accuracy: 0.8323 - val_loss: 0.4273 - val_accuracy: 0.8123
Epoch 6/10
215/215 [==============================] - 2s 11ms/step - loss: 0.3805 - accuracy: 0.8316 - val_loss: 0.4286 - val_accuracy: 0.8176
Epoch 7/10
215/215 [==============================] - 2s 12ms/step - loss: 0.3743 - accuracy: 0.8358 - val_loss: 0.4272 - val_accuracy: 0.8176

In [68]:
pred_data_6_1 = tf.squeeze(tf.round(model_6_1.predict(val_sent)))
model_6_1_results = calculate_results(val_labels, pred_data_6_1)
model_6_1_results

24/24 [==============================] - 1s 9ms/step


{'Accuracy': 82.93963254593176,
 'Precision': 0.8318659277497434,
 'Recall value': 0.8293963254593176,
 'F1-Score': 0.8280362455483953}